In [1]:
Vir = lie_algebras.VirasoroAlgebra(QQ)
M = Vir.verma_module(1/2, 1/2)
d = Vir.basis()
v = M.highest_weight_vector()
m2 = (-4*d[-2] + 3*d[-1]*d[-1])*v
m3 = (-12*d[-2]*d[-1] + 4*d[-1]*d[-1]*d[-1] + 3*d[-3])*v
var('t q')

def L_partition_vector(partition, vector):
    for i in range(len(partition)):
        vector = d[-partition[len(partition) - i - 1]]*vector
    return vector

def my_coefficient(partition, vector):
    return vector.monomial_coefficients().get(tuple(-k for k in partition), 0)

def number_of_ones(partition):
    for i in range(len(partition)):
        if partition[len(partition) - i - 1] != 1:
            return i
    return len(partition)

def my_len(partition):
    return 2*len(partition) - number_of_ones(partition)

def row_to_vector(row, n):
    list_of_partition = ordered_partitions(n)
    return sum(row[i]*L_partition_vector(list_of_partition[i], v) for i in range(len(row)))

def series_t_q(list_of_list_of_partition):
    series = 0
    for i in range(len(list_of_list_of_partition)):
        series_t = 0
        for j in range(len(list_of_list_of_partition[i])):
            series_t = series_t + t^my_len(list_of_list_of_partition[i][j])
        series = series + series_t*q^i
    return series

def partition_ends(partition, tail, is_greater):
    if len(partition) < len(tail) - 1:
        return False
    partition2 = partition[len(partition) - len(tail) + 1: len(partition)]
    if partition2 != tail[1:]:
        return False
    if is_greater:
        if partition == partition2:
            return True
        if partition[len(partition) - len(tail)] > tail[0]:
            return True
    else:
        if partition == partition2:
            return False
        if partition[len(partition) - len(tail)] == tail[0]:
            return True
    return False

def list_of_list_of_partition_ends(list_of_list_of_partition, tail, is_greater):
    return [[partition
             for partition in list_of_partition if partition_ends(partition, tail, is_greater)]
            for list_of_partition in list_of_list_of_partition]

def partition_to_polynomial(partition):
    polynomial = [0]*(sum(partition) - 1)
    for i in range(len(partition) - number_of_ones(partition)):
        polynomial[partition[i] - 2] += 1
    return polynomial

def my_order(partition):
    polynomial = partition_to_polynomial(partition)
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((my_len(partition), len(partition) - number_of_ones(partition), polynomial)) 

def ordered_partitions(n):
    if n <= 0:
        return [[]]
    list_of_partition = list(Partitions(n))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def is_subpartition(partition, partition2):
    if len(partition) < len(partition2):
        return False
    for i in range(len(partition) - len(partition2) + 1):
        if partition[i: i + len(partition2)] == partition2:
            return True
    return False

def partition_contains_any_of_list(partition, list_of_partition):
    for partition2 in list_of_partition:
        if is_subpartition(partition, partition2):
            return True
    return False

def partition_contains_any_of_Q(partition):
    if partition == []:
        return False
    if partition_contains_any_of_list(
        partition,
        [[2], [1, 1, 1], [3, 1, 1], [3, 3], [4, 3, 1], [4, 4, 1], [5, 4, 1, 1], [6, 5, 3, 1]]):
        return True
    if len(partition) > 2:
        for i in range(len(partition) - 2):
            if partition[i] - partition[i + 2] <= 2:
                return True
    if len(partition) > 3:
        for i in range(len(partition) - 3):
            if (partition[i] - partition[i + 3] <= 3
                or (partition[i] - partition[i + 3] <= 4
                    and partition[i + 1] - partition[i + 2] <= 3)):
                return True
    if len(partition) > 4:
        for i in range(len(partition) - 4):
            if (partition[i] - partition[i + 1] == 1 and partition[i + 1] - partition[i + 2] == 2
                and partition[i + 2] - partition[i + 3] == 2
                and partition[i + 3] - partition[i + 4] == 1):
                return True
    return False

def matrix_UVir_m2_m3(n):
    list_of_partition = ordered_partitions(n)
    list_of_partition2 = ordered_partitions(n - 2)
    list_of_partition3 = ordered_partitions(n - 3)
    matrix_m2 = matrix({
        (i, j): my_coefficient(list_of_partition[j], L_partition_vector(list_of_partition2[i], m2))
        for j in range(len(list_of_partition))
        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m3 = matrix({
        (i, j): my_coefficient(list_of_partition[j], L_partition_vector(list_of_partition3[i], m3))
        for j in range(len(list_of_partition))
        for i in range(len(list_of_partition3))}, sparse = False)
    matrix_m2 = matrix_m2.stack(matrix_m3)
    matrix_m2 = matrix_m2.rref()
    return matrix_m2

def mW_partition(partition):
    n = sum(partition)
    k = ordered_partitions(n).index(partition)
    A_n = matrix_UVir_m2_m3(n)
    pivots = A_n.pivots()
    try:
        return row_to_vector(A_n.row(pivots.index(k)), n)
    except ValueError:
        return 0

def mW_list_of_partition(list_of_partition):
    return [mW_partition(partition) for partition in list_of_partition]

def pivots_up_to(n):
    list_of_list_of_partition = []
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [list_of_partition[j] for j in matrix_UVir_m2_m3(i).pivots()])
    return list_of_list_of_partition

def basis_up_to(n):
    list_of_list_of_partition = []
    pivots = pivots_up_to(n)
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition if partition not in pivots[i]])
    return list_of_list_of_partition

def basis_up_tov2(n):
    list_of_list_of_partition = []
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition
             if not partition_contains_any_of_Q(partition)])
    return list_of_list_of_partition

In [2]:
gr_G_M = CombinatorialFreeModule(QQ, Partitions(NonNegativeIntegers()));
B = gr_G_M.basis()

m2 = B[Partition([2])] - 3/4*B[Partition([1, 1])]
m3 = B[Partition([1, 1, 1])]
m5 = B[Partition([3, 1, 1])]
m6 = B[Partition([3, 3])] + 1/3*B[Partition([4, 1, 1])]
m8 = B[Partition([4, 3, 1])]
m9 = B[Partition([4, 4, 1])] + 9/8*B[Partition([5, 3, 1])]
m11 = B[Partition([5, 4, 1, 1])]
m15 = B[Partition([6, 5, 3, 1])]
a = B[Partition([2, 2, 2])]
b = B[Partition([4, 3, 2])] + 1/6*B[Partition([5, 2, 2])]

my_list_of_element = [m2, m3, m5, m6, m8, m9]
my_list_of_element2 = [a, b]

def number_of_ones(a):
    k = 0
    for i in range(len(a)):
        if a[i]==1:
            k = k+1
    return k

def to_dict(element):
    return {tuple(term[0]): term[1] for term in element}

def prt2(a):
    st = ""
    m = number_of_ones(a)
    for i in range(len(a)-m):
        st = st + "x(" + str(a[i]) + ")*"
    st = st + "gen(" + str(m+1) + ")"
    return st

def prt1(u):
    st = ""
    for a in set(u):
        st = st + str(u[a]) + "*" + prt2(a) + "+"
    st = st[:-1]
    return st

def prt(w):
    st = ""
    for i in range(len(w)):
        st = st + prt1(to_dict(w[i])) + ","
    st = st[:-1]
    return st

def to_polynomial(a,n):
    b = [0]*n
    b[n-1] = number_of_ones(a)
    for i in range(len(a)):
        if a[i]!= 1:
            b[a[i]-2] = b[a[i]-2]+1
    return b

def p_divides_q(p,q,n):
    for i in range(n):
        if p[i]>q[i]:
            return false
    return true

def a_contains_any_of(a, avoid, n):
    p = to_polynomial(a, n)
    if p[n-1]>2:
        return true
    for i in range(len(avoid)):
        if p[n-1] == avoid[i][n-1] and p_divides_q(avoid[i][:-1], p[:-1], n-1):
            return true
    return false

def weight(element):
    for term in element:
        return sum(list(term[0]))

def partial0(partition):
    return sum((partition[k] - 1)*B[Partition(sorted(partition[: k] + [partition[k] + 1] + partition[k + 1:], reverse = True))] for k in range(len(partition)))

def partial(element):
    return sum(term[1]*partial0(list(term[0])) for term in element)

def poisson_bracket0(partition):
    return B[Partition(partition + [1])]

def poisson_bracket(element):
    return sum(term[1]*poisson_bracket0(list(term[0])) for term in element)

def differential_ideal_up_to(list_of_element, n):
    list_of_element2 = []
    for element in list_of_element:
        list_of_element3 = []
        m = n - weight(element)
        if m >= 0:
            list_of_element3 = [element]
            for k in range(m):
                list_of_element3.append(partial(list_of_element3[-1]))
        list_of_element2 += list_of_element3
    return list_of_element2

def poisson_submodule_up_to(list_of_element, n):
    list_of_element2 = []
    for element in list_of_element:
        list_of_element3 = []
        m = n - weight(element)
        if m >= 0:
            list_of_element3 = [element]
            for k in range(m):
                list_of_element3.append(poisson_bracket(list_of_element3[-1]))
        list_of_element2 += list_of_element3
    return list_of_element2

def poisson_submodule_and_differential_ideal_up_to(list_of_element, list_of_element2, n):
    return poisson_submodule_up_to(list_of_element + differential_ideal_up_to(list_of_element2, n), n)

def avoid_up_to(n):
    w = poisson_submodule_and_differential_ideal_up_to(my_list_of_element, my_list_of_element2, n)
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(2..' + str(n) + ')), (dp, C);'
    st = st + 'module M =' + prt(w) + ';'
    st = st + 'module G = std(M);'
    st = st + 'string result = "";'
    st = st + 'for(int i = 1; i<=size(G); i++){result = result + "[" + string(leadexp(G[i])) + "],";}'
    st = st + 'result = result[1..size(result)-1];'
    st = st + 'result = "[" + result + "]";'
    st = st + 'result;'
    st = singular.eval(st)
    avoid = eval(st)
    for i in range(len(avoid)):
        avoid[i][n-1] = avoid[i][n-1]-1
    return avoid

def m11_m15_in_submodule(n):
    w = poisson_submodule_and_differential_ideal_up_to(my_list_of_element, my_list_of_element2, n)
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(2..' + str(n) + ')), (dp, C);'
    st = st + 'module M =' + prt(w) + ';'
    st = st + 'module G = std(M);'
    st = st + 'vector f = x(5)*x(4)*gen(3);'
    st = st + 'vector g = x(6)*x(5)*x(3)*gen(2);'
    st = st + '(reduce (f, G), reduce (g, G));'
    st = singular.eval(st)
    return st

def basis_up_tov3(n):
    base = []
    avoid = avoid_up_to(n)
    for i in range(n+1):
        add = []
        a = ordered_partitions(i)
        for j in range(len(a)):
            if not a_contains_any_of(a[j], avoid, n):
                add = add + [a[j]]
        base = base + [add]
    return base

%time series_t_q(basis_up_tov3(25))

CPU times: user 700 ms, sys: 60.2 ms, total: 760 ms
Wall time: 12.1 s


(3*t^9 + 24*t^8 + 22*t^7 + 35*t^6 + 10*t^5 + 11*t^4 + t^3 + t^2)*q^25 + (t^9 + 19*t^8 + 19*t^7 + 32*t^6 + 9*t^5 + 11*t^4 + t^3 + t^2)*q^24 + (13*t^8 + 16*t^7 + 28*t^6 + 9*t^5 + 10*t^4 + t^3 + t^2)*q^23 + (9*t^8 + 14*t^7 + 25*t^6 + 8*t^5 + 10*t^4 + t^3 + t^2)*q^22 + (5*t^8 + 11*t^7 + 22*t^6 + 8*t^5 + 9*t^4 + t^3 + t^2)*q^21 + (3*t^8 + 9*t^7 + 19*t^6 + 7*t^5 + 9*t^4 + t^3 + t^2)*q^20 + (t^8 + 7*t^7 + 16*t^6 + 7*t^5 + 8*t^4 + t^3 + t^2)*q^19 + (5*t^7 + 14*t^6 + 6*t^5 + 8*t^4 + t^3 + t^2)*q^18 + (3*t^7 + 11*t^6 + 6*t^5 + 7*t^4 + t^3 + t^2)*q^17 + (2*t^7 + 9*t^6 + 5*t^5 + 7*t^4 + t^3 + t^2)*q^16 + (7*t^6 + 5*t^5 + 6*t^4 + t^3 + t^2)*q^15 + (5*t^6 + 4*t^5 + 6*t^4 + t^3 + t^2)*q^14 + (3*t^6 + 4*t^5 + 5*t^4 + t^3 + t^2)*q^13 + (2*t^6 + 3*t^5 + 5*t^4 + t^3 + t^2)*q^12 + (3*t^5 + 4*t^4 + t^3 + t^2)*q^11 + (2*t^5 + 4*t^4 + t^3 + t^2)*q^10 + (t^5 + 3*t^4 + t^3 + t^2)*q^9 + (3*t^4 + t^3 + t^2)*q^8 + (2*t^4 + t^3 + t^2)*q^7 + (t^4 + t^3 + t^2)*q^6 + (t^3 + t^2)*q^5 + (t^3 + t^2)*q^4 + q^3*t^2 + q^2*

In [3]:
m11_m15_in_submodule(15)

'0 0'

In [4]:
print(my_list_of_element)
print(my_list_of_element2)
prt(poisson_submodule_and_differential_ideal_up_to(my_list_of_element, my_list_of_element2, 15))

[-3/4*B[[1, 1]] + B[[2]], B[[1, 1, 1]], B[[3, 1, 1]], B[[3, 3]] + 1/3*B[[4, 1, 1]], B[[4, 3, 1]], B[[4, 4, 1]] + 9/8*B[[5, 3, 1]]]
[B[[2, 2, 2]], B[[4, 3, 2]] + 1/6*B[[5, 2, 2]]]


'-3/4*gen(3)+1*x(2)*gen(1),-3/4*gen(4)+1*x(2)*gen(2),1*x(2)*gen(3)+-3/4*gen(5),-3/4*gen(6)+1*x(2)*gen(4),1*x(2)*gen(5)+-3/4*gen(7),-3/4*gen(8)+1*x(2)*gen(6),-3/4*gen(9)+1*x(2)*gen(7),-3/4*gen(10)+1*x(2)*gen(8),1*x(2)*gen(9)+-3/4*gen(11),-3/4*gen(12)+1*x(2)*gen(10),-3/4*gen(13)+1*x(2)*gen(11),1*x(2)*gen(12)+-3/4*gen(14),1*x(2)*gen(13)+-3/4*gen(15),-3/4*gen(16)+1*x(2)*gen(14),1*gen(4),1*gen(5),1*gen(6),1*gen(7),1*gen(8),1*gen(9),1*gen(10),1*gen(11),1*gen(12),1*gen(13),1*gen(14),1*gen(15),1*gen(16),1*x(3)*gen(3),1*x(3)*gen(4),1*x(3)*gen(5),1*x(3)*gen(6),1*x(3)*gen(7),1*x(3)*gen(8),1*x(3)*gen(9),1*x(3)*gen(10),1*x(3)*gen(11),1*x(3)*gen(12),1*x(3)*gen(13),1/3*x(4)*gen(3)+1*x(3)*x(3)*gen(1),1*x(3)*x(3)*gen(2)+1/3*x(4)*gen(4),1/3*x(4)*gen(5)+1*x(3)*x(3)*gen(3),1/3*x(4)*gen(6)+1*x(3)*x(3)*gen(4),1/3*x(4)*gen(7)+1*x(3)*x(3)*gen(5),1/3*x(4)*gen(8)+1*x(3)*x(3)*gen(6),1*x(3)*x(3)*gen(7)+1/3*x(4)*gen(9),1/3*x(4)*gen(10)+1*x(3)*x(3)*gen(8),1/3*x(4)*gen(11)+1*x(3)*x(3)*gen(9),1/3*x(4)*gen(12)+1*x(3)*